This notebbok generates:
1. Movie clips with keyboints, bounding boxes, trackers and classes
2. A dictionary on the format {'Aaron': [dists, frames], ..., 'video': 2t_t9}

In [1]:
# Imports
import sys  
sys.path.insert(0, 'C:\\Users\\espebh\\Documents\\Thesis\\code_ver2')
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision
import math
import pickle
import os
import math
import datetime
import cv2
from helpers import m
from helpers.formatting_functions import cv2_to_keyrcnn_in
from helpers.video_monitor import VideoMonitor



In [2]:
# Specify the paths where the different outputs should be saved
vid_in_path = r'C:\\Users\\espebh\\Documents\\Thesis\\data\\Results\\Results from full pipeline in tank 6\\salmon_jaw_distances_061022_t6_c1_f2000\\source videos'
vid_out_path = r'C:\\Users\\espebh\\Documents\\Thesis\\data\\Results\\Results from full pipeline in tank 6\\salmon_jaw_distances_061022_t6_c1_f2000\\generated videos'
tracker_out_path = r'C:\\Users\\espebh\\Documents\\Thesis\\data\\Results\\Results from full pipeline in tank 6\\salmon_jaw_distances_061022_t6_c1_f2000\\trackers'

keyrcnn_man_label_path = r'C:\\Users\\espebh\\Documents\\Thesis\\data\\keyrcnn_200_dataset\\manually annotated data'
keyrcnn_model_path = r'C:\\Users\\espebh\\Documents\\Thesis\\data\\models\\keyrcnn\\13_epoch_6000_dataset'

In [3]:
# Load models
keyrcnnmodel = torchvision.models.detection.keypointrcnn_resnet50_fpn(weights=None,
                                                               weights_backbone=torchvision.models.ResNet50_Weights.IMAGENET1K_V1,
                                                               num_keypoints=m.NUM_KEYPOINTS,
                                                               num_classes = m.NUM_CLASSES_T9)
keyrcnnmodel.load_state_dict(torch.load(keyrcnn_model_path, map_location=m.DEVICE))
keyrcnnmodel.to(m.DEVICE).eval()

toraxIDmodel = torchvision.models.resnet101(pretrained=True)
toraxIDmodel.fc = nn.Linear(in_features=2048, out_features=m.NUM_CLASSES_T6)
toraxIDmodel.load_state_dict(torch.load('C:\\Users\\espebh\\Documents\\Thesis\\data\\models\\tank_6\\finetuning_resnet101_d226k_8epoch', map_location=m.DEVICE))
toraxIDmodel.to(m.DEVICE).eval()

c:\Users\espebh\Documents\Thesis\code_ver2\dat_gen_th\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\espebh\Documents\Thesis\code_ver2\dat_gen_th\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# Specify the video bounds
start_frame = 5000
num_frames_in_video = 2000
flip = False
#ann_img_path = 'C:\\Users\\espebh\\Documents\\Thesis\\data\\Temporary workspace\\images\\annotated_frames'


In [5]:
for vid_name in os.listdir(vid_in_path):

    if vid_name.split('.')[0][-1] == 'r':
       flip = True
    else:
       flip = False

    video_in = cv2.VideoCapture(os.path.join(vid_in_path, vid_name))
    print("Number of frames in input video: " + str(video_in.get(cv2.CAP_PROP_FRAME_COUNT)))
    video_in.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Generate movie name
    mov_txt = vid_name.split('.')[0] + '_f_' + str(start_frame) + '_' + str(start_frame + num_frames_in_video) + '_'
    date = datetime.datetime.now()
    ID = str(date.year) + str(date.month) + str(date.day) + str(date.hour) + str(date.minute)
    movie_name = mov_txt + ID

    video_out = cv2.VideoWriter(os.path.join(vid_out_path, movie_name + '.avi'), cv2.VideoWriter_fourcc(*'mp4v'), 100.0, (1920, 1080), True)
    print('Saving movie as: ' + movie_name)

    VM = VideoMonitor(toraxIDmodel)

    # Create a video with keypoint and bbox annotations
    while True:
        suc, img = video_in.read() # Read video frame
        if suc:
            # cv2 to tensor
            img_cv2 = img.copy()
            if flip: 
                #img_cv2 =  cv2.rotate(img_cv2, cv2.ROTATE_180)
                img_cv2 =  cv2.rotate(img_cv2, cv2.ROTATE_180)
            img_keyrcnn_in = cv2_to_keyrcnn_in(img_cv2.copy())
            
            # Pass frame through neural network
            output = keyrcnnmodel([img_keyrcnn_in])
            VM.update_trackers(output[0])
            VM.update_classes_and_dists(img_cv2.copy(), video_in.get(cv2.CAP_PROP_POS_FRAMES))
            img_ann = VM.draw_active_tracker(img_keyrcnn_in, classes_map=m.CLASSES_MAP_T6)

            # Store image to video and disk
            video_out.write(np.uint8(img_ann))
            frame = str(video_in.get(cv2.CAP_PROP_POS_FRAMES)).split('.')[0]

            # Store annotated image
            #cv2.imwrite(os.path.join(ann_img_path, frame + '.jpeg'), img_ann)
        else:
            # If the capture object cannot read a frame, exit loop
            break

        # Print number of frames
        if int(video_in.get(cv2.CAP_PROP_POS_FRAMES)) % 10 == 0:
            print("Number of frames: " + str(video_in.get(cv2.CAP_PROP_POS_FRAMES)))

        # Exit loop if we have enough frames
        if video_in.get(cv2.CAP_PROP_POS_FRAMES) > start_frame + num_frames_in_video:
            break

    # Release capture object and windows
    video_in.release()
    video_out.release()

    log_file_trackers = open(os.path.join(tracker_out_path, movie_name + '.pkl'), 'wb')
    pickle.dump(VM.log_trackers+VM.active_trackers, log_file_trackers)
    log_file_trackers.close()

    print('The video ', movie_name, ' was successfully saved')

print('All movies finished')

Number of frames in input video: 42420.0
Saving movie as: 061022_t6_c1_1t_unc_s1_f_5000_7000_20235301038
Number of frames: 5010.0
Number of frames: 5020.0
Number of frames: 5030.0
Number of frames: 5040.0
Number of frames: 5050.0
Number of frames: 5060.0
Number of frames: 5070.0
Number of frames: 5080.0
Number of frames: 5090.0
Number of frames: 5100.0
Number of frames: 5110.0
Number of frames: 5120.0
Number of frames: 5130.0
Number of frames: 5140.0
Number of frames: 5150.0
Number of frames: 5160.0
Number of frames: 5170.0
Number of frames: 5180.0
Number of frames: 5190.0
Number of frames: 5200.0
Number of frames: 5210.0
Number of frames: 5220.0
Number of frames: 5230.0
Number of frames: 5240.0
Number of frames: 5250.0
Number of frames: 5260.0
Number of frames: 5270.0
Number of frames: 5280.0
Number of frames: 5290.0
Number of frames: 5300.0
Number of frames: 5310.0
Number of frames: 5320.0
Number of frames: 5330.0
Number of frames: 5340.0
Number of frames: 5350.0
Number of frames: 53